#DMC DDS SFA16_TAREA FINAL#
* Se les entrega este notebook para que lo usen como referencia y pueda darles idea de cómo abordar su propio planteamiento.
* Deben generar su propio planteamiento y sus propios análisis. Se calificará la problemática planteada y los comentarios de sus resulatados según los vayan obteniendo.
Nota.- No se calificará aquellos trabajos que sean sean réplicas de este notebook o que no hayan realizado sus propios plnateamientos.


# Sección nueva

In [ ]:

#### Importamos las librerías a usar
import numpy as np
import pandas as pd
import glob
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


In [ ]:
### llamamos a nuestros datos ubicados dentro de la carpeta de trabajo en Jupyter lab
#file = glob.glob('*.xlsx')
#file

#['DMC Análisis de datos_ENAHO 2020_Mod400_VF.xlsx']

### Leemos nuestros datos
df = pd.read_excel('data.xlsx')


In [ ]:
df.columns.tolist()

Estamos considerando esta columnas para nuestro analisis

In [ ]:
columnas_deseadas = ['GASTO TOTAL','P203','P208A','P207','P301A','P4192','AREA']
columnas_presentes = [col for col in columnas_deseadas if col in df.columns]
df = df[columnas_presentes]

In [ ]:
df.head(10)

In [ ]:
# Seleccionar solo columnas categóricas (object o category)
categorical_cols = df.select_dtypes(include=['object', 'category'])

# Obtener los valores únicos por columna categórica
unique_values = {col: df[col].unique().tolist() for col in categorical_cols}

# Mostrar los resultados
for col, values in unique_values.items():
    print(f"{col}: {values}")

In [ ]:
missing_values = df.isna().sum()
missing_values

In [ ]:
df.columns.tolist()

**Filtros al consideracion**



Se considero nulo el campo P401 debido a que en el campo tipo de cuestionario todos los registros fueron realizados desde una tablet , esto puede sugerir que el presente  campo no haya sido obligatorio


In [ ]:
df= df.dropna(subset=['P401'])

In [ ]:
nulos = df['P401'].isna().sum()
print("Verificamos si hay un dato vacion en columna P401-->>",nulos)


Se eliminan los registros vacios del campo "NIVEL" debido a que el campo "TIPOCUESTIONARIO" tambien se encuentra vacio , lo que indica que no se tiene conocimiento del medio por el que fue realizado el cuestionario,ademas , los gastos fueron nulos

In [ ]:
df= df.dropna(subset=['NIVEL'])

In [ ]:
nulos = df['NIVEL'].isna().sum()
print("Verificamos si hay un dato vacion en columnaNIVEL-->>",nulos)


In [ ]:
df.head(10)

Se tomo en consideracion los campos P401H1,P401H2,P401H3,P401H4,P401H5,P401H6 debido a que indican que la persona que realizo el cuestionario puede contar con algún tipo de discapacidad, dato considerado relevante para el monto gastado en servicios de salud .

In [ ]:
df['Discapacidad'] = df[['P401H1', 'P401H2', 'P401H3', 'P401H4', 'P401H5', 'P401H6']].apply(lambda row: 1 if 'Si' in row.values else 0, axis=1)


In [ ]:
df['Discapacidad'].value_counts()

In [ ]:
df.head(10)

In [ ]:
#procedemos en eliminar las columnas que hemos usado para el campo calculado de discapacidad
df = df.drop(['P401H1', 'P401H2', 'P401H3', 'P401H4', 'P401H5', 'P401H6'], axis=1)

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
print(df.dtypes)

In [ ]:
#analizamos nuestro target
#ubicamos los valores atipicos a no considerar
import seaborn as sns

sns.boxplot(y=df['GASTO TOTAL'])

In [ ]:
df.query('`GASTO TOTAL` > 2500').head(10)

Se eliminaran los registros que cuentan con datos mayores a 2500 debido a que son datos atipicos,esto pudo ocurrir debido a que los registros fueron realizados a travez de una tablet y la persona encuestada pudo haber confundido los puntos decimales

In [ ]:
filtered_data = df[(df['GASTO TOTAL'] <= 2000)]
sns.boxplot(y=filtered_data['GASTO TOTAL'])


Se consideran 5118 registros con monto 0 debido a que existe la posibilidad de que el encuestado haya considerado sus gastos solo de las ultimas 4 semanas . Esto pudo ocurrir porque una de las preguntas que incluye gastos en medicamentos en general solo abarca el lead time de 4 semanas

In [ ]:
cantidad_ceros = (filtered_data['GASTO TOTAL'] == 0).sum()
print(cantidad_ceros)
filtered_data['GASTO TOTAL'].count()

In [ ]:
filtered_data['GASTO TOTAL'].describe()

In [ ]:
df['P401'] = df['P401'].replace({'Si': 1, 'No': 0})


Analizamos las variables para asignar el target y nuestras variables



[texto del enlace](https://)

In [ ]:
# Target y variables
variables = ['P203','P207','P208A','P209','P301A','NIVEL','TICUEST01A','AREA','PROV','DISTRITO'  ]

target = 'P401'

targetlINEAL = 'GASTO TOTAL'

In [ ]:
df['P208A'].describe()

In [ ]:
# Distribución de variables contínuas

def plot_distribution(df, variable):
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(df[variable], kde=True, bins=30, color='skyblue')
    ax.set_title(f'Distribution of {variable}', fontsize=16)
    ax.set_xlabel(variable, fontsize=14)
    ax.set_ylabel('Frequency', fontsize=14)
    plt.show()
plot_distribution(df, 'P208A')

In [ ]:
def plot_box(df, variable):
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.boxplot(x=df[variable], color='skyblue')
    ax.set_title(f'Box Plot of {variable}', fontsize=16)
    ax.set_xlabel(variable, fontsize=14)
    plt.show()
plot_box(df, 'P208A')

#.3 Division dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[variables ], df[target], test_size=0.3, random_state=42, stratify= df[target])
D_Train = pd.concat([X_train,y_train], axis=1)
D_Test = pd.concat([X_test,y_test], axis=1)

In [ ]:
X_train.head(10)

In [ ]:
y_train.head(10)

In [ ]:
D_Train.columns

In [ ]:
D_Train.head(10)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df[variables], df[target], test_size=0.3, random_state=42)
#D_Train = pd.concat([X_train,y_train], axis=1)
#D_Test = pd.concat([X_test,y_test], axis=1)

In [ ]:
df.head(10)

In [ ]:
import matplotlib.pyplot as plt

# Contar la cantidad de cada valor en la columna 'P301A'
conteo_valores = df['P301A'].value_counts()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
conteo_valores.plot(kind='bar', color='skyblue')

# Añadir etiquetas y título
plt.xlabel('Valores en la columna P301A')
plt.ylabel('Cantidad')
plt.title('Distribución de valores en la columna P301A')

# Mostrar las cantidades encima de las barras
for index, value in enumerate(conteo_valores):
    plt.text(index, value, str(value), ha='center', va='bottom')

# Mostrar el gráfico
plt.show()


In [ ]:
columnas = ['P401', 'P203', 'P207', 'P208A', 'P209', 'P301A', 'NIVEL', 'TICUEST01A', 'AREA', 'PROV', 'DISTRITO', 'GASTO TOTAL', 'Discapacidad']

# Contar los valores vacíos en cada columna
valores_vacios = df[columnas].isna().sum()

# Imprimir los resultados
print(valores_vacios)


## Feature Engineering

In [ ]:
# Imputacion de datos
#feature_impute_median = ['INGRESO_ESTIMADO']
feature_impute_mode = ['P209']
feature_replace_outliers = ['P208A']
feature_woe_transform = [ 'P203', 'P207', 'P301A', 'NIVEL', 'TICUEST01A', 'AREA', 'PROV', 'DISTRITO', 'Discapacidad']

In [ ]:
# Outliers
''' La variable edad(P208A) tiene valores poco razonables; como edad menor a 21 o mayor a 98.
Imputaremos outliers con valores extremos según el IQR. Para el valor inferior asumimos que la edad mínima
para abrir un depósito a plazo es de 18'''

variable = 'P208A'
edad_min = 21
edad_max = np.percentile(D_Train['P208A'], 75) + 3 * (np.percentile(D_Train['P208A'], 75) - np.percentile(D_Train['P208A'], 25))

In [ ]:
D_TrainF = D_Train[(D_Train['P208A']>=edad_min)&
                    (D_Train['P208A']<=edad_max)]

print('Cantidad de Registros sin Outliers: '+str(D_TrainF.shape[0]))
D_TrainF.head()

In [ ]:
D_TrainF.info()

In [ ]:
D_TrainF['P209'].describe()

In [ ]:
D_TrainF['P209'] = D_TrainF['P209'].fillna('Casado(a)')

In [ ]:
D_TrainF['P301A'] = D_TrainF['P301A'].replace('99','Sin Nivel')

In [ ]:
D_TrainF.info()

In [ ]:
D_Train.columns

In [ ]:
D_Train.info()

In [ ]:
def woe_analysis(data, analysis_type, analysis_columns):
    list1 = []
    list2 = []
    tabla_f = pd.DataFrame()
    data[target] = data[target].astype(int)
    for columnName in analysis_columns:
        x = data[columnName].values
        y = data[target]
        optb = OptimalBinning(name=columnName, dtype=analysis_type, solver="cp")
        optb.fit(x, y)
        binning_table = optb.binning_table

        tabla = binning_table.build()
        b = tabla[tabla.index == 'Totals']['IV'][0]
        list1.append(b)
        list2.append(columnName)
        tabla_ = tabla[['Bin', 'WoE']].iloc[:-3]
        tabla_['variable'] = columnName
        tabla_f = pd.concat([tabla_f, tabla_], axis=0)
    return tabla_f

In [ ]:
categorical_var = [  'P203', 'P207',  'P209', 'P301A', 'NIVEL', 'TICUEST01A',
       'AREA', 'PROV', 'DISTRITO']

numerical_var = ['P208A']

In [ ]:
!pip install optbinning

In [ ]:
# Importación de librerías
# Manejo de data
import pandas as pd
import numpy as np
import os
import re

# Analisis de datos
from optbinning import OptimalBinning, BinningProcess
# Modelamiento
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import pickle
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, roc_auc_score, recall_score, precision_score, f1_score, roc_curve, auc

from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd

# Supongamos que ya tienes tu DataFrame D_TrainF
# Filtrar la fila con índice 4020
fila_5020 = D_TrainF.loc[5420]
print(fila_5020)


In [ ]:
D_TrainF.columns

In [ ]:
woe_categorical_vars = woe_analysis(D_TrainF, 'categorical', categorical_var)
mapeo_por_variable = {}
grupos = woe_categorical_vars.groupby('variable')

# Recorre cada grupo y crea un diccionario de mapeo para cada variable
for variable, grupo in grupos:
    mapeo = {}
    for index, row in grupo.iterrows():
        for bin_value in row['Bin']:
            mapeo[bin_value] = row['WoE']
    mapeo_por_variable[variable] = mapeo

In [ ]:
# Función para aplicar woe
def woe_transform(data_cat, columnas_interes, dic_woe):
    for columna in columnas_interes:
        data_cat['woe_' + columna] = data_cat[columna].map(dic_woe.get(columna, {}))
    columns = ['woe_' + k for k in columnas_interes]
    return data_cat[columns]

woe_transformer = FunctionTransformer(func=woe_transform, kw_args={'columnas_interes': feature_woe_transform, 'dic_woe': mapeo_por_variable})

In [ ]:
woe_transformer

## **Analizamos nuestra regresion logistica**




---



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### REGRESIÓN LOGÍSTICA ###
#En su archivo de datos excel de ENAHO 400 quédese solo con las variables que va a utilizar para el modelo de RLogística
#Y=	P401	¿Padece de alguna enfermedad o malestar crónico (Artritis, hipertensión, asma, reumatismo, diabetes, tuberculosis, VIH, colesterol, etc.)?
# X1=	GASTO TOTAL
# X2=	P207	Sexo
# X3=	P208A	¿Qúe edad tiene en años cumplidos? (En años)
# X4=	P209	¿Cuál es su estado civil o conyugal?
# X5=	P301A	¿Cuál es el último año o grado de estudios y nivel que aprobó? - Nivel
# X6=	AREA	Área urbana o rural
#### renombre el archivo si es necesario
# Creamos nuestras variables
#X = np.array(df.drop(['P401'],axis=1))
#y = np.array(df['P401'])
#X.shape

In [ ]:
#vamos usar los mismos datos de entrenamiento de del anterior modelo


In [ ]:
D_TrainF.head()

In [ ]:
df_Final = df.copy()

In [ ]:
df_Final.head(10)

In [ ]:
D_TrainF['P301A'] = D_TrainF['P301A'].replace('99','Sin Nivel')

In [ ]:
print(D_TrainF['P301A'].unique())

In [ ]:
D_TrainF['P301A'].mode()

In [ ]:
#D_Test1['P301A'].mode()

In [ ]:
D_TrainF['P301A'].describe()

In [ ]:
D_TrainF['P301A'] = D_TrainF['P301A'].fillna('Secundaria Completa')


In [ ]:
# Convertir la columna P301A a numérica usando map

D_TrainF['P301A'] = D_TrainF['P301A'].map({
    'Superior Universitaria Incompleta': 0,
    'Secundaria Completa': 1,
    'Primaria Incompleta': 2,
    'Superior Universitaria Completa': 3,
    'Secundaria Incompleta': 4,
    'Superior No Universitaria Completa': 5,
    'Sin Nivel': 6,
    'Superior No Universitaria Incompleta': 7,
    'Inicial': 8,
    'Maestria/Doctorado': 9,
    'Primaria Completa': 10,
    '99': 11,
    'Básica especial': 12 })


In [ ]:
# Convertir variables categóricas a numéricas

D_TrainF['P207'] = D_TrainF['P207'].map({'Hombre': 0, 'Mujer': 1})  # Ejemplo
D_TrainF['P209'] = D_TrainF['P209'].fillna('Desconocido')
D_TrainF['P209'] = D_TrainF['P209'].map({'Casado(a)': 0,
    'Soltero(a)': 1,
    'Viudo(a)': 2,
    'Conviviente': 3,
    'Separado(a)': 4,
    'Divorciado(a)': 5,
    'Desconocido':6                             })  # Ejemplo
D_TrainF['AREA'] = D_TrainF['AREA'].map({'URBANO': 0, 'RURAL': 1})  # Ejemplo

In [ ]:
D_TrainF.head(10)

In [ ]:
# Filtrar las filas donde P301A tiene valores nulos
filas_nulas111 = D_TrainF[D_TrainF['P301A'].isnull()]

# Mostrar las filas con nulos en P301A
print(filas_nulas111)


In [ ]:
columnasV = ['P401', 'P203', 'P207', 'P208A', 'P209', 'P301A', 'NIVEL', 'TICUEST01A', 'AREA', 'PROV', 'DISTRITO']

for columnasV in columnasV:
    nulos = D_TrainF[columnasV].isnull().sum()
    print(f"Columna {columnasV}: {nulos} valores faltantes")


In [ ]:
sns.heatmap(D_TrainF.corr(numeric_only = True), annot=True)


In [ ]:
 sns.pairplot(D_TrainF)

*LIMPIESA* TEST


In [ ]:
D_Test['P301A'].mode()
#Secundaria Completa
D_Test['P301A'] = D_Test['P301A'].fillna('Secundaria Completa')
# Convertir la columna P301A a numérica usando map

D_Test['P301A'] = D_Test['P301A'].map({
    'Superior Universitaria Incompleta': 0,
    'Secundaria Completa': 1,
    'Primaria Incompleta': 2,
    'Superior Universitaria Completa': 3,
    'Secundaria Incompleta': 4,
    'Superior No Universitaria Completa': 5,
    'Sin Nivel': 6,
    'Superior No Universitaria Incompleta': 7,
    'Inicial': 8,
    'Maestria/Doctorado': 9,
    'Primaria Completa': 10,
    '99': 11,
    'Básica especial': 12 })




# Convertir variables categóricas a numéricas

D_Test['P207'] = D_Test['P207'].map({'Hombre': 0, 'Mujer': 1})  # Ejemplo
D_Test['P209'] = D_Test['P209'].fillna('Desconocido')
D_Test['P209'] = D_Test['P209'].map({    'Casado(a)': 0,
    'Soltero(a)': 1,
    'Viudo(a)': 2,
    'Conviviente': 3,
    'Separado(a)': 4,
    'Divorciado(a)': 5,
    'Desconocido':6                             })  # Ejemplo
D_Test['AREA'] = D_Test['AREA'].map({'URBANO': 0, 'RURAL': 1})  # Ejemplo

columnasV = ['P401', 'P203', 'P207', 'P208A', 'P209', 'P301A', 'NIVEL', 'TICUEST01A', 'AREA', 'PROV', 'DISTRITO']

for columnasV in columnasV:
    nulos = D_Test[columnasV].isnull().sum()
    print(f"Columna {columnasV}: {nulos} valores faltantes")

In [ ]:
D_Test.head(10)

In [ ]:
D_TrainF1 = D_TrainF[['P207', 'P208A', 'P209', 'P301A', 'AREA']]

In [ ]:
D_TrainF2 =  D_TrainF[['P401']]

In [ ]:
D_TrainF.head(10)

In [ ]:
# Crear el modelo
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo
model.fit(D_TrainF1, D_TrainF2 )


In [ ]:
D_Test.head(10)

In [ ]:
D_Test1 = D_Test[['P207', 'P208A', 'P209', 'P301A', 'AREA']]

In [ ]:
D_Test2 = D_TrainF[[ 'P401']]

[texto del enlace](https://)## **Analisis Regresion Lineal**

In [ ]:
df["GASTO TOTAL"].describe()

In [ ]:
sns.histplot(filtered_data["GASTO TOTAL"]
, kde=True)

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x='GASTO TOTAL', data=filtered_data);

In [ ]:
filtered_data['GASTO TOTAL'].describe()

In [ ]:
filtered_data.head(10)

In [ ]:
#ajuste de modelo de regresion
#Podemos visualizar en una parte el modelo sumary estadistica , el siguiente es variables de sumatoria, residual(error) de estadistica
X = sm.add_constant(filtered_data["P208A"])
y = filtered_data["GASTO TOTAL"]

model = sm.OLS(y, X).fit()

model.summary()

#podemos interpretar que un aumento de la edad persona, aumentaria el Gasto Total de atencion
#y = 73 + x

In [ ]:
model.predict([1,2])

In [ ]:
newEnaho = pd.DataFrame({'P208A':[0,.5,.8,1,2,3,4,5,6,7,8]})

In [ ]:
model.predict(sm.add_constant(newEnaho))

In [ ]:
#evaluamos el valor p de nuestra regresion simple
#F-statistic:	86.79
#Prob (F-statistic):	1.77e-20
#abarcamos como p es muy cercano a cero y es menos que alfa 0.05
#se rechaza la hipotesis nul
#podemos determinar que la edad del encuestado es buen predictor de acuerdo al gasto total#

#estamos determinando ademas que p tanto constante y edad son ceros porque p es menor que alfa 0.05

In [ ]:
#ahora analizamos los residuos de la regrersion logistica modelo star models
residuals = pd.DataFrame(
    {
       "Edad" :filtered_data["P208A"],
       "Gasto Total":filtered_data["GASTO TOTAL"],
       "Prediced Gasto Total" : model.predict(),
       "Residuos":model.resid

    }
)
residuals.head(10)

In [ ]:
sns.scatterplot(x="Prediced Gasto Total", y="Residuos", data=residuals)

In [ ]:
#verificamos que tenemos 2 variables numericas el resto son categoricos
sns.pairplot(filtered_data)

In [ ]:
sns.lmplot(filtered_data, x="P208A", y="GASTO TOTAL",hue="NIVEL")

In [ ]:
sns.lmplot(filtered_data, x="P208A", y="GASTO TOTAL",hue="P401")

In [ ]:
sns.pairplot(filtered_data, corner=True, kind="kde")

In [ ]:
sns.heatmap(filtered_data.corr(numeric_only = True), annot=True)

In [ ]:
import numpy as np
import pandas as pd
import glob
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#### modelo de regresión lineal simple
####### P208A EDAD###
dataX =filtered_data[["P208A"]]
X_train = np.array(dataX)
y_train = filtered_data['GASTO TOTAL'].values
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
LinearRegression()
y_pred = regr.predict(X_train)

*4*.	Realice un análisis de regresión simple con las variables que seleccionó anteriormente.

In [ ]:
# Calculando los estimadores de la regresión lineal
# Veamos los coeficienetes obtenidos, En nuestro caso, serán la Tangente
print('Coefficients: \n', regr.coef_)
# Este es el valor donde corta el eje Y (en X=0)
print('Independent term: \n', regr.intercept_)
# Error Cuadrado Medio
print("Mean squared error: %.2f" % mean_squared_error(y_train, y_pred))
# Puntaje de Varianza. El mejor puntaje es un 1.0
print('Variance score: %.2f' % r2_score(y_train, y_pred))

In [ ]:
# Vamos a realizar predicciones
# según nuestro modelo, hacemos:
y_25Anios = regr.predict([[25]])
print(int(y_25Anios))

In [ ]:
# según nuestro modelo, hacemos:
y_50Anios = regr.predict([[50]])
print(int(y_50Anios))

In [ ]:
# según nuestro modelo, hacemos:
y_80Anios = regr.predict([[80]])
print(int(y_80Anios))